# Делегаты

**Делегаты** являются реализацией механизма *обратного вызова* в C# и CLR.

Делегат - тип, экземпляры которого способны хранить и вызывать методы с фиксированным набором параметров и типом возвращаемого значения.

 ## 1. Пример

In [ ]:
// Пример объявления делегата
public delegate double BinaryOperation(double left, double right);

In [ ]:
double CalculateMark(double accumulated, double exam)
{
    if(exam < 4.0)
    {
        return exam;
    }
    return 0.4 * exam + 0.6 * accumulated; 
}

In [ ]:
int[] GetResultMarks((double Accumulated, double Exam)[] studentMarks, BinaryOperation formula)
{
    int[] result = new int[studentMarks.Length];
    if (formula is null) 
    {
        return result;
    }
    for(int i = 0; i < studentMarks.Length; i++)
    {
        // Тут на самом деле вызывается formula.Invoke(...)
        double formulaResult = formula(studentMarks[i].Accumulated, studentMarks[i].Exam);
        result[i] = (int)Math.Round(formulaResult, MidpointRounding.AwayFromZero);
    }
    return result;
}

In [ ]:
var marks = new (double Accumulated, double Exam)[] { (3.7, 6.5), (9.8, 7.4), (8.4, 9.2) };

// На самом деле это разворачивается в GetResultMarks(marks, new BinaryOperation(CalculateMark))
GetResultMarks(marks, CalculateMark)

## 2. Внутренности

Любой делегат под капотом представляет из себя специальный класс. Например, для делегата
```csharp
public delegate double BinaryOperation(double left, double right);
```
на этапе компиляции создастся примерно такой класс:

In [ ]:
internal class BinaryOperation : System.MulticastDelegate
{
    public BinaryOperation(Object obj, IntPtr method);
    
    public virtual double Invoke(double left, double right);
    
    public virtual IAsyncResult BeginInvoke(double left, double right, AsyncCallback callback, Object object);
    
    public virtual void EndInvoke(IAsyncResult result);
}

Из этого полезно знать только о `Invoke`, т.к. можно вызывать делегаты безопасно

In [ ]:
Action veryGoodAction = null;

veryGoodAction()

In [ ]:
veryGoodAction?.Invoke()

## 3. Ковариантность и контравариантность

Делегаты поддерживают ковариацию и контравариацию **для ссылочных типов** при привязке методов к делегату.

**Ковариантность** позволяет привязать к делегату метод с типом возвращаемого значения, *производного* от типа, возвращаемого делегатом.

**Контравариантность** позволяет привязать к делегату метод с типом параметра, *базового* для типа параметра делегата.

Пример ковариации и контравариации в одном флаконе:

In [ ]:
using System.IO;

// Обратите внимание на типы
public delegate object MyCallback(string s);

public class MyClass
{
    // Обратите внимание на типы
    public string SomeMethod(object s)
    {
        // do something
        
        return string.Empty;
    }
}

In [ ]:
MyClass obj = new MyClass();

// Успешно компилируется
MyCallback callback = obj.SomeMethod;

**За этим стоит следующая логика:**

Делегат возвращает object => метод возвращает string => string это object => можно засунуть этот метод в делегат. \[*ковариантность*\]

При вызове в делегат будет передан FileStream => метод принимает Stream => FileStream это Stream => можно засунуть этот метод в делегат. \[*контравариантность*\]

## 4. Из каждого экземпляра делегата можно достать:
- Target - объект, к которому применяется сохранённый метод, null если метод статический;
- Method - объект типа MethodInfo, представляющий информацию о методе.

In [ ]:
callback.Target == obj // Это и есть ссылка на исходный объект

In [ ]:
var methodInfo = callback.Method;
methodInfo.Name

## 5. Цепочки делегатов

В один делегат можно запихнуть несколько делегатов, они вызовутся в порядке добавления. Если делегат имеет возвращаемое значение, вернётся результат последнего метода.

In [ ]:
public delegate int MyCallback();

public static int Oh()
{
    Console.WriteLine("Oh");
    return 1;
}

public int My()
{
    Console.WriteLine("My");
    return 2;
}

public int God()
{
    Console.WriteLine("God");
    return 3;
}

In [ ]:
MyCallback callback = Oh;
callback += My; // Под капотом: callback = (MyCallback)Delegate.Combine(callback, My);
callback += God;

int returnValue = callback();

returnValue

Можно "отменять подписку". Удаляются делегаты **с конца**.

In [ ]:
MyCallback callback;
callback += Oh;
callback += My;
callback += Oh;
callback += My;

callback -= Oh;

callback -= God; // Если метода не было в списке, ничего не произойдёт

callback()

## 6. Action и Func

В реальности новые типы делегатов объявлять не приходится, т.к. существуют обобщённые делегаты System.Action и System.Func, которые покрывают все потребности.

Их объявления выглядят примерно так:

In [ ]:
public delegate void Action<T1, T2, ... T16>(T1 arg1, T2 arg2, ... T16 arg16);

public delegate TResult Func<T1, T2, ... T16, TResult>(T1 arg1, T2 arg2, ... T16 arg16);

In [ ]:
public static string Repeat(string str, int ntimes) => string.Concat(Enumerable.Repeat(str, ntimes));

Func<string, int, string> myFunc = Repeat;

myFunc("abcd", 10)

In [ ]:
public static void SayHello(int ntimes)
{
    for(int i = 0; i < ntimes; i++)
    {
        Console.WriteLine("Hello!");
    }
}

In [ ]:
Action<int> action = SayHello;

action(3)

# Лямбда-выражения

Причина введения лямбда выражений - громоздкий синтаксис использования делегатов. Зачастую мы хотим передать небольшую функцию, которая не имеет смысла вне контекста.

In [ ]:
// Анонимная функция - старый синтаксис из мезозоя (до C# 2.0)
Func<int, int, bool> dontWriteLikeThis = delegate(int left, int right) { 
    double dist = Math.Abs(right - left);
    return dist > 5;
};

Func<int, int, bool> func1 = (int left, int right) => { 
    double dist = Math.Abs(right - left);
    return dist > 5;
};

// Если компилятор сам может определить типы, их можно не писать
Func<int, int, bool> func2 = (left, right) => Math.Abs(right - left) > 5;

In [ ]:
// Для одного параметра можно опустить скобки
Action<string> print = str => Console.WriteLine(str);

print("Hello world")

Скобки можно опустить только когда у нас 1 параметр без явного указания типа:

In [ ]:
// Можно
Action<string> print1 = str => Console.WriteLine(str);

In [ ]:
// Неможно
Action<string> print2 = string str => Console.WriteLine(str);

In [ ]:
// Неможно
Action<string, string> print3 = str1, str2 => Console.WriteLine(str1 + str2);

Лямбды нельзя присваивать неявно типизированным переменным. Впрочем, обычно вы определяете лямбду прямо при вызове метода, так что это не нужно.

In [ ]:
var doesntCompile = (int left, int right) => Math.Abs(right - left) > 5;

### Захват переменных (замыкание)

In [ ]:
Action action;

for (int i = 0; i < 5; i++)
{
    action += () => Console.WriteLine(i);
}

action.Invoke()

# События

- Сущность
- Допустимые операции
- Методы аксессоры
- Общепринятые правила использования
- Пример

## 1. Сущность

Событие (event) - обёртка над делегатом, предоставляющая более безопасное использование.

Объявляется как обычный член класса, например:

In [ ]:
public class EventHolder
{
    public event EventHandler<EventArgs> MyUselessEvent;
}

## 2. Допустимые операции

Допустимые операции **извне класса**:
- Подписать метод на событие (+=);
- Отписать метод от события (-=);

Всё. Нельзя **вызвать событие** или **присвоить ему новое значение** - это прерогатива самого класса. Это и отличает события от делегатов. 

In [ ]:
public class EventHolder
{
    public event EventHandler<EventArgs> MyUselessEvent;
    
    public void TriggerEvent(EventArgs e)
    {
        MyUselessEvent?.Invoke(this, e);
    }
}

In [ ]:
EventHolder ev = new EventHolder();

ev.MyUselessEvent += (sender, e) => Console.WriteLine("Hello");
ev.MyUselessEvent += (sender, e) => Console.WriteLine("World");

// Не получится изменить значение
// ev.MyUselessEvent = (sender, e) => Console.WriteLine("Goodbye");

// Не получится вызвать напрямую
// ev.MyUselessEvent?.Invoke()

ev.TriggerEvent(EventArgs.Empty)

## 3. Методы аксессоры

Свойство - эдакое свойство, но для делегатов. 

В реальности событие разворачивается в приватное поле делегата и 2 метода **add** и **remove**, соответствующие операциям "+=" и "-=" соответственно.

- Их можно переопределить, но как правило это не требуется.
- Вы не можете переопределить один - нужны оба!
- **Переопределение add и remove отнимает возможность вызывать событие "напрямую"!**

In [ ]:
public class EventHolder
{
    private EventHandler<EventArgs> myEvent; 
    
    public event EventHandler<EventArgs> MyUselessEvent
    {
        add 
        { 
            myEvent += value;
            Console.WriteLine("Adding new method"); 
        }
        remove 
        { 
            myEvent -= value;
            Console.WriteLine("Removing method"); 
        }
    }
    
    public void TriggerEvent(EventArgs e)
    {
        // Так нельзя, раз переопределены аксессоры!
        // MyUselessEvent?.Invoke(this, e);
        
        myEvent?.Invoke(this, e);
    }
}

In [ ]:
EventHolder ev = new EventHolder();

ev.MyUselessEvent += (sender, e) => Console.WriteLine("Do nothing"); 
ev.MyUselessEvent -= (sender, e) => Console.WriteLine("Anyway this is not working");

ev.TriggerEvent(EventArgs.Empty);

## 4. Общепринятые правила использования

**Invocation list**, который в делегатах применяется не очень часто, очень полезен в событиях: предполагается, что на событие может подписаться много методов. Поэтому мы не хотим создавать события с возвращаемым значением, отличным от void - в случае вызова нескольких методов мы получим результат лишь последнего.

Можно использовать одну из вариаций Action, но классическим считается использование делегата EventHandler<TEventArgs>, сигнатура которого выглядит так:

In [ ]:
public delegate void EventHandler<TEventArgs>(object sender, TEventArgs e);

Ограничений на ```TEventArgs``` не накладывается, хотя могли бы. Предполагается, что будет использоваться наследник EventArgs, примерно так:

In [ ]:
public class MessageEventArgs : EventArgs
{
    public string Message { get; set; }
}

In [ ]:
public class EventHolder
{
    public event EventHandler<MessageEventArgs> MyUselessEvent;
    
    public void TriggerEvent(MessageEventArgs e)
    {
        MyUselessEvent?.Invoke(this, e);
    }
}

In [ ]:
EventHolder ev = new EventHolder();

ev.MyUselessEvent += (sender, e) => Console.WriteLine("1. " + e.Message); 
ev.MyUselessEvent += (sender, e) => Console.WriteLine("2. " + e.Message);

ev.TriggerEvent(new MessageEventArgs { Message = "You do not talk about Fight Club" });